In [2]:
from pathlib import Path
import sys
import subprocess
import shutil
from datetime import datetime

# Folder where manage.py lives
PROJECT_DIR = Path(r"/Users/puttisanaekkapornpisan/Projects/ez_ram")
MANAGE_PY = PROJECT_DIR / "manage.py"

# Directory containing JSON files
JSON_DIR = Path(r"/Users/puttisanaekkapornpisan/Projects/ez_ram/question_json")

# Create organization folders
ALREADY_IMPORTED_DIR = JSON_DIR / "already_imported"
ERROR_IMPORT_DIR = JSON_DIR / "error_import"

# Create folders if they don't exist
ALREADY_IMPORTED_DIR.mkdir(exist_ok=True)
ERROR_IMPORT_DIR.mkdir(exist_ok=True)

print(f"📁 Organization folders ready:")
print(f"   ✓ {ALREADY_IMPORTED_DIR}")
print(f"   ✓ {ERROR_IMPORT_DIR}\n")

# Get all JSON files in the main directory (excluding subdirectories)
json_files = sorted([f for f in JSON_DIR.glob("*.json") if f.is_file()])
total_files = len(json_files)

if total_files == 0:
    print(f"No JSON files found in {JSON_DIR}")
    print(f"All files may have already been processed.")
else:
    print(f"Found {total_files} JSON file(s) to process\n")
    
    # Track statistics
    successful_count = 0
    error_count = 0
    skipped_count = 0
    
    for index, json_file in enumerate(json_files, start=1):
        print(f"\n{'='*60}")
        print(f"Processing file {index} of {total_files}: {json_file.name}")
        print(f"{'='*60}\n")
        
        # Set INPUT_DIR to current file
        INPUT_DIR = json_file
        
        cmd = [
            sys.executable,          # use the current Python (works with your venv)
            str(MANAGE_PY),
            "create_question_graph",
            "--file",
            str(INPUT_DIR),
            "--yes",  # Skip confirmation prompt
        ]
        
        try:
            # Run with the working dir set to the project so manage.py resolves settings
            subprocess.run(cmd, check=True, cwd=PROJECT_DIR)
            print(f"\n✓ Successfully processed: {json_file.name}")
            
            # Move file to already_imported folder
            destination = ALREADY_IMPORTED_DIR / json_file.name
            shutil.move(str(json_file), str(destination))
            print(f"📦 Moved to: already_imported/{json_file.name}")
            successful_count += 1
            
        except subprocess.CalledProcessError as e:
            print(f"\n✗ Error processing {json_file.name}")
            print(f"   Error details: {e}")
            
            # Move file to error_import folder
            destination = ERROR_IMPORT_DIR / json_file.name
            shutil.move(str(json_file), str(destination))
            print(f"📦 Moved to: error_import/{json_file.name}")
            
            # Create error log file
            error_log_path = ERROR_IMPORT_DIR / f"{json_file.stem}_error.log"
            with open(error_log_path, 'w') as log_file:
                log_file.write(f"Error Log for: {json_file.name}\n")
                log_file.write(f"Timestamp: {datetime.now().isoformat()}\n")
                log_file.write(f"{'='*60}\n\n")
                log_file.write(f"Command executed:\n")
                log_file.write(f"{' '.join(cmd)}\n\n")
                log_file.write(f"Error:\n")
                log_file.write(f"{str(e)}\n\n")
                log_file.write(f"Return code: {e.returncode}\n")
            print(f"📝 Error log created: {error_log_path.name}")
            error_count += 1
            
            print("\nDo you want to continue to the next file despite the error? (y/n): ", end="")
            error_response = input().strip().lower()
            if error_response not in ['y', 'yes']:
                print("\nStopping processing due to error.")
                skipped_count = total_files - index
                break
            continue
        
        # Ask user if they want to continue (skip for the last file)
        if index < total_files:
            print(f"\nFile '{json_file.name}' processed.")
            print(f"Do you want to continue to the next file? (y/n): ", end="")
            response = input().strip().lower()
            
            if response not in ['y', 'yes']:
                print(f"\nStopping at user request.")
                skipped_count = total_files - index
                break
        else:
            print(f"\n{'='*60}")
            print(f"All {total_files} files have been processed!")
            print(f"{'='*60}")
    
    # Print summary statistics
    print(f"\n\n{'='*60}")
    print(f"PROCESSING SUMMARY")
    print(f"{'='*60}")
    print(f"✓ Successfully imported: {successful_count} file(s)")
    print(f"✗ Failed imports: {error_count} file(s)")
    if skipped_count > 0:
        print(f"⊘ Skipped: {skipped_count} file(s)")
    print(f"Total processed: {successful_count + error_count} of {total_files}")
    print(f"{'='*60}")
    print(f"\n📂 Check folders:")
    print(f"   • already_imported/ - {successful_count} file(s)")
    print(f"   • error_import/ - {error_count} file(s)")
    if skipped_count > 0:
        print(f"   • question_json/ - {skipped_count} unprocessed file(s) remaining")


📁 Organization folders ready:
   ✓ /Users/puttisanaekkapornpisan/Projects/ez_ram/question_json/already_imported
   ✓ /Users/puttisanaekkapornpisan/Projects/ez_ram/question_json/error_import

Found 1 JSON file(s) to process


Processing file 1 of 1: example_quiz.json

Fetching knowledge nodes from Neo4j...


WARNING Failed to fetch knowledge nodes: Couldn't connect to 192.168.1.40:7687 (resolved to ()):
Timed out trying to establish connection to ResolvedIPv4Address(('192.168.1.40', 7687))
INFO Sending 10 questions in ONE batch AI request


Fetched 0 knowledge nodes.
Loaded 10 items from /Users/puttisanaekkapornpisan/Projects/ez_ram/question_json/example_quiz.json.
Using batch processing (batch_size=10, parallelism=10, rps=2.0, knowledge_nodes=0)
Processing batch 1 (questions 1-10)...


INFO HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO Received 10 mappings from AI in single response


  ✓ Completed 10/10 questions

Predictions written for review:
  predictions/example_quiz_mappings.json


Traceback (most recent call last):
  File "/Users/puttisanaekkapornpisan/Projects/ez_ram/.venv/lib/python3.11/site-packages/neo4j/_async_compat/network/_bolt_socket.py", line 528, in _connect
    s.connect(resolved_address)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/puttisanaekkapornpisan/Projects/ez_ram/.venv/lib/python3.11/site-packages/neo4j/_async_compat/network/_bolt_socket.py", line 690, in connect
    s = BoltSocket._connect(resolved_address, tcp_timeout,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/puttisanaekkapornpisan/Projects/ez_ram/.venv/lib/python3.11/site-packages/neo4j/_async_compat/network/_bolt_socket.py", line 537, in _connect
    raise ServiceUnavailable(
neo4j.exceptions.ServiceUnavailable: Timed out trying to establish connection to ResolvedIPv4Address(('192.168.1.40', 7687))

The above exception was the direct cause of the following ex


✗ Error processing example_quiz.json
   Error details: Command '['/Users/puttisanaekkapornpisan/Projects/ez_ram/.venv/bin/python', '/Users/puttisanaekkapornpisan/Projects/ez_ram/manage.py', 'create_question_graph', '--file', '/Users/puttisanaekkapornpisan/Projects/ez_ram/question_json/example_quiz.json', '--yes']' returned non-zero exit status 1.
📦 Moved to: error_import/example_quiz.json
📝 Error log created: example_quiz_error.log

Do you want to continue to the next file despite the error? (y/n): 
Stopping processing due to error.


PROCESSING SUMMARY
✓ Successfully imported: 0 file(s)
✗ Failed imports: 1 file(s)
Total processed: 1 of 1

📂 Check folders:
   • already_imported/ - 0 file(s)
   • error_import/ - 1 file(s)
